### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import time 

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

### Getting target list 

In [2]:
dataalbums = pd.read_csv('../data/clean/api_albums_clean.csv', encoding='latin1')

In [3]:
dataalbums.head(2)

,albumnamers,albumidspotify,albumnamespotify,artistnamespotify,artistidspotify,releasedate,totaltracks
0,Sgt. Pepper's Lonely Hearts Club Band,6QaVfG1pHYl1z15ZxkvVDW,Sgt. Pepper's Lonely Hearts Club Band (Remaste...,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,1967-06-01,13.0
1,Pet Sounds,6GphKx2QAPRoVGWE9D7ou8,Pet Sounds (Original Mono & Stereo Mix),The Beach Boys,3oDbviiivRWhXwIE8hxkVV,1966-06-16,27.0


In [4]:
artist_ids = dataalbums['artistidspotify'].unique().tolist()

In [5]:
artist_ids[0:2]

['3WrFJ7ztbogyGnTHbHJFl2', '3oDbviiivRWhXwIE8hxkVV']

### Spotify Connection 

In [6]:
#%pip install pyarrow
#%pip install joblib
#%pip install tqdm
#%pip install spotipy
#%pip install python-dotenv
#%pip install requests 

In [7]:
import requests as rq
import pyarrow
from joblib import Parallel, delayed
from tqdm import tqdm
import pymongo
from passwords import *

In [8]:
# Obtener el TOKEN

AUTH_URL = "https://accounts.spotify.com/api/token"

creds = {
    "grant_type": "client_credentials",
    "client_id": Client_ID,
    "client_secret": Client_secret
}

response = rq.post(AUTH_URL, 
                         data=creds, 
                         headers={"Content-Type": "application/x-www-form-urlencoded"})


TOKEN = response.json()['access_token']

# Conexión con la API

url = "https://api.spotify.com/"

headers = {"Authorization": f'Bearer {TOKEN}'}

response = rq.get(url,headers=headers)
response

<Response [200]>

### Getting Tracks Information 

In [9]:
album_ids = dataalbums['albumidspotify'].unique().tolist()

In [10]:
album_ids[0:3]

['6QaVfG1pHYl1z15ZxkvVDW', '6GphKx2QAPRoVGWE9D7ou8', '3PRoXYsngSwjEQWR5PsHWR']

In [11]:
#endpoint
#https://api.spotify.com/v1/albums/{id}/tracks

In [12]:
#test 
url = 'https://api.spotify.com/v1/albums/1x1jpjDbetGqX0IKCUIBNj/tracks'

In [13]:
response = rq.get(url, headers=headers)

In [14]:
responsejson = response.json()

In [15]:
type(responsejson)

dict

In [16]:
responsejson.keys()

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [18]:
responsejson['items'][0]['name']

"Sgt. Pepper's Lonely Hearts Club Band"

In [19]:
# Construct a list of URLs for querying track information
track_urls = [f'https://api.spotify.com/v1/albums/{album_id}/tracks' for album_id in album_ids]

# Create a list of dictionaries with album information and corresponding URLs
track_url_list = [{'album_id': album_id, 'url': url} for album_id, url in zip(album_ids, track_urls)]

In [21]:
# List to store the retrieved track data
track_data_list = []

# Function to search and retrieve track information
def search_track_info(track_url):
    try:
        response = rq.get(track_url['url'], headers=headers)
        response.raise_for_status()  # Check for request errors
        data = response.json()

        for track in data['items']:
            track_data_list.append({
                'track_id': track['id'],
                'track_name': track['name'],
                'album_id': track_url['album_id'],
                'artist_id': track['artists'][0]['id'],
                'artist_name': track['artists'][0]['name']
                # You can add more data points here that you want to fetch directly
            })

    except rq.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except Exception as e:
        print(f"An error occurred: {e}")
        # Handle the error as needed

# Execute the function for each track URL
for track_url in track_url_list:
    search_track_info(track_url)

# Create a DataFrame from the list of dictionaries
data_tracks = pd.DataFrame(track_data_list)

HTTP error occurred: 400 Client Error: Bad Request for url: https://api.spotify.com/v1/albums/nan/tracks


In [26]:
data_tracks.head(5)

,track_id,track_name,album_id,artist_id,artist_name
0,4fUKE8EULjQdHF4zb0M8FO,Sgt. Pepper's Lonely Hearts Club Band - Remast...,6QaVfG1pHYl1z15ZxkvVDW,3WrFJ7ztbogyGnTHbHJFl2,The Beatles
1,2RnPATK99oGOZygnD2GTO6,With A Little Help From My Friends - Remastere...,6QaVfG1pHYl1z15ZxkvVDW,3WrFJ7ztbogyGnTHbHJFl2,The Beatles
2,25yQPHgC35WNnnOUqFhgVR,Lucy In The Sky With Diamonds - Remastered 2009,6QaVfG1pHYl1z15ZxkvVDW,3WrFJ7ztbogyGnTHbHJFl2,The Beatles
3,3LtOmWpTXLhilL5odoKysR,Getting Better - Remastered 2009,6QaVfG1pHYl1z15ZxkvVDW,3WrFJ7ztbogyGnTHbHJFl2,The Beatles
4,3pKKxkeB1pOUMHwWBmKc3Y,Fixing A Hole - Remastered 2009,6QaVfG1pHYl1z15ZxkvVDW,3WrFJ7ztbogyGnTHbHJFl2,The Beatles


In [27]:
data_tracks.shape

(6222, 5)

In [28]:
data_tracks.to_csv('../data/clean/api_tracks1_clean.csv', index=False)